In [ ]:
import os
import glob
import numpy as np
from numpy import ones
import pandas as pd
#import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
import pathlib
import hashlib
import yaml
import re
import pdb
import logging
from tqdm import tqdm


dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt


# Custom utils
from utils.utils_data import *
from utils.utils_ml import *
from utils.utils_resnet import *
from utils.utils_plot import *
from utils.utils_unet import *
from utils.utils_RF import create_xarray_frompred


In [ ]:
import yaml
conf = yaml.safe_load(open("config.yaml"))

In [ ]:
# Load data
lons_x = np.load('tmp/data/lons_y.npy')
lats_y = np.load('tmp/data/lats_y.npy')
# load predictors
dg_train_X = np.array(xr.open_dataarray('tmp/data/dg_train_X.nc'))
dg_test_X = np.array(xr.open_dataarray('tmp/data/dg_test_X.nc'))
# load amount of precipitation
dg_train_Y = np.array(xr.open_dataarray('tmp/data/dg_train_Y.nc')).squeeze()
dg_test_Y = np.array(xr.open_dataarray('tmp/data/dg_test_Y.nc')).squeeze()
# load extremes
dg_train_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_train_Y_xtrm0.95th.nc')).squeeze()
dg_test_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_test_Y_xtrm0.95th.nc')).squeeze()
# load percentiles
qq = np.load('tmp/data/qq_095.npy')

In [ ]:
times = np.arange(np.datetime64('2016-01-01'), np.datetime64('2022-01-01'))
#times = pd.to_datetime(times)

In [ ]:
# Still need to add RF
models_report = ['RF','Pan-orig','UNET2','RaNet']

In [ ]:
def create_xarray_frompred(preds, time, lats_y, lons_x):
    """Function to create the xarray 3D of predictions from the outputs from the xr.apply_ufunc
       Args: preds are the prediction for each grid cell that contains the output values"""
    # create the xarray of predictions
    mx= xr.DataArray(np.zeros((time, len(lats_y),len(lons_x))), dims=["time","lat", "lon"],
                  coords=dict(lat = lats_y, 
                  lon = lons_x))
    # put the outputs for each latitude and longitue, 
    for ilat in range(len(lats_y)):
        for ilon in range(len(lons_x)):
            if preds[ilat,ilon] is None:
                mx[:,ilat,ilon] = np.nan
            else:
                mx[:,ilat,ilon] = preds[ilat, ilon]

    #pred_matrix.rename({'dim_0':'time', 'dim_1': 'lat','dim_2': 'lon'})  

    return(mx)

In [ ]:
# Store the mean values in a table
df = pd.DataFrame(columns = ['model', 'cor', 'IoU', 'Iou_xtrm'])

In [ ]:
xr_true = convert_to_xarray(dg_test_Y, lats_y, lons_x, times)

n_rows = len(models_report)
fig, axs = plt.subplots(n_rows, 3, figsize=(12, n_rows*3))

for idx, m_id in enumerate(models_report):
    print(m_id)
    
    pred_test = np.load(f'tmp/data/predictions/y_pred_test_{m_id}.npy',allow_pickle=True)
    # for extremes
    #pred_train_xtrm = np.load(f'tmp/data/predictions/y_pred_train_xtrm_{m}.npy')
    pred_test_xtrm = np.load(f'tmp/data/predictions/y_pred_test_xtrm_{m_id}.npy', allow_pickle=True)
    
    if (m_id == 'RF'):
        pred_test = create_xarray_frompred(pred_test, len(times), lats_y, lons_x)
        pred_test_xtrm = create_xarray_frompred(pred_test_xtrm, len(times), lats_y, lons_x)
    
    y_pred_bool = pred_test > qq
    # Multiply to transform to numeric values
    y_pred_bool = y_pred_bool * 1
    # for the correlation
    xr_pred = convert_to_xarray(pred_test, lats_y, lons_x, times)
   

    cc = xr.apply_ufunc(
        get_corrcoef,    # First the function
        xr_true, xr_pred,        # then the input data arrays
        input_core_dims=[['time'], ['time']],
        vectorize=True)

    test_iou=IoU(dg_test_Y_xtrm, y_pred_bool)

    
  
    prec_xtrm = np.where(pred_test_xtrm > 0.5, 1, 0)
    test_iou_xtrm=IoU(dg_test_Y_xtrm, prec_xtrm)
    
    # Plotting for each DNN-as Fig.1
    
    plot_map(axs[idx,0], lons_x, lats_y, cc, vmin = 0, vmax = 1, cmap = "RdBu_r", title=f"Correlation - {m_id}")
    plot_map(axs[idx,1], lons_x, lats_y, test_iou, vmin = 0, vmax = 100, title=f"IoU - {m_id}")
    plot_map(axs[idx,2], lons_x, lats_y, test_iou_xtrm, vmin = 0, vmax = 100, title=f"IoU - {m_id}")
    
    # add in the table
    df= df.append({'model': m_id}, ignore_index=True)
    df.at[df.index[-1], 'cor'] = np.nanmean(cc)
    df.at[df.index[-1], 'IoU'] = np.nanmean(test_iou)
    df.at[df.index[-1], 'Iou_xtrm'] = np.nanmean(test_iou_xtrm)

fig.tight_layout()
plt.savefig('figures/corr_patterns.pdf')

In [ ]:
df